<a href="https://colab.research.google.com/github/geordee/colab/blob/main/HRMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HRMS Data Analytics
Analyze HRMS data using Python and Pandas
Pandas user guide is available at<br />
https://pandas.pydata.org/docs/user_guide/index.html#user-guide

Import required data analysis libraries

In [1]:
!pip install -q xlrd
import numpy as np
import pandas as pd


Read Excel file data as a Dataframe

In [2]:
employee_df = pd.read_excel("HRMS.xlsx", sheet_name="Employees")
employee_df.head()

,EmployeeID,Name,Department,Band,JoinDate
0,16000325,Zorita Evans,Accounting,1,2001-04-06
1,16001028,Daryl Hendricks,Human Resources,2,2003-12-05
2,16001128,Bruno Gallegos,Quality Assurance,2,2017-01-25
3,16011224,Yael Cannon,Quality Assurance,5,2004-04-21
4,16040206,Nathan Malone,Payroll,3,2002-04-06


Add JoinYear as a new column using a transformation on JoinDate

In [3]:
employee_df['JoinDate'] = pd.to_datetime(employee_df['JoinDate'])
employee_df['JoinYear'] = employee_df['JoinDate'].dt.year
employee_df.head()

,EmployeeID,Name,Department,Band,JoinDate,JoinYear
0,16000325,Zorita Evans,Accounting,1,2001-04-06,2001
1,16001028,Daryl Hendricks,Human Resources,2,2003-12-05,2003
2,16001128,Bruno Gallegos,Quality Assurance,2,2017-01-25,2017
3,16011224,Yael Cannon,Quality Assurance,5,2004-04-21,2004
4,16040206,Nathan Malone,Payroll,3,2002-04-06,2002


Pivot table on Department and Band, and aggregate (count)

In [4]:
employee_df.pivot_table(index=['Department'], columns='Band', values='EmployeeID', aggfunc='count').fillna('')

Band,1,2,3,4,5,6
Department,,,,,,
Accounting,3.0,1.0,1.0,2.0,,2.0
Advertising,,1.0,,2.0,1.0,
Asset Management,,2.0,2.0,3.0,,
Customer Relations,1.0,1.0,5.0,2.0,,
Customer Service,1.0,1.0,2.0,1.0,,1.0
Finances,,1.0,1.0,4.0,,
Human Resources,,3.0,1.0,,1.0,
Legal Department,1.0,1.0,1.0,3.0,2.0,
Media Relations,,2.0,2.0,2.0,,


Filter the table and pivot

In [5]:
employee_df[employee_df.Department == "Research and Development"].pivot_table(index=['Department'], columns='Band', values='EmployeeID', aggfunc='count').fillna(0)

Band,1,2,3,4
Department,,,,
Research and Development,2,1,3,2


## Compute Bonus

In [6]:
bonus_df = employee_df.loc[:, ["EmployeeID"]]
bonus_df.head()

,EmployeeID
0,16000325
1,16001028
2,16001128
3,16011224
4,16040206


In [7]:
salary_df = pd.read_excel("HRMS.xlsx", sheet_name="Salaries")
salary_df.head()

,EmployeeID,Salary
0,16000325,2000
1,16240817,2000
2,16150729,2000
3,16400329,2187
4,16041013,3102


In [8]:
bonus_df = pd.merge(bonus_df, 
                    salary_df[['EmployeeID','Salary']], 
                    on = 'EmployeeID', 
                    how = 'inner')
bonus_df.head()

,EmployeeID,Salary
0,16000325,2000
1,16001028,7194
2,16001128,4965
3,16011224,15134
4,16040206,10614


In [9]:
bonus_df = pd.merge(bonus_df, 
                    employee_df[['EmployeeID','Band']], 
                    on = 'EmployeeID', 
                    how = 'inner')
bonus_df.head()

,EmployeeID,Salary,Band
0,16000325,2000,1
1,16001028,7194,2
2,16001128,4965,2
3,16011224,15134,5
4,16040206,10614,3


In [10]:
ratings_df = pd.read_excel("HRMS.xlsx", sheet_name="Ratings")
ratings_df.head()

,EmployeeID,Rating
0,16691225,3
1,16000325,4
2,16890922,5
3,16121207,4
4,16590222,4


In [11]:
bonus_df = pd.merge(bonus_df, 
                    ratings_df[['EmployeeID','Rating']], 
                    on = 'EmployeeID', 
                    how = 'inner')
bonus_df.head()

,EmployeeID,Salary,Band,Rating
0,16000325,2000,1,4
1,16001028,7194,2,5
2,16001128,4965,2,3
3,16011224,15134,5,3
4,16040206,10614,3,4


In [12]:
settings_df = pd.read_excel("HRMS.xlsx", sheet_name="Settings")
settings_df.head()

,Band,Rating,Bonus %
0,1,1,1.0
1,2,1,1.0
2,3,1,0.8
3,4,1,0.8
4,5,1,0.8


In [13]:
bonus_df = pd.merge(bonus_df, 
                    settings_df[['Band','Rating','Bonus %']], 
                    on = ['Band','Rating'] ,
                    how = 'inner')
bonus_df.head()

,EmployeeID,Salary,Band,Rating,Bonus %
0,16000325,2000,1,4,1.2
1,16041013,3102,1,4,1.2
2,16150625,5283,1,4,1.2
3,16150729,2000,1,4,1.2
4,16240817,2000,1,4,1.2


In [14]:
bonus_df['Bonus']=bonus_df['Salary']*2*bonus_df['Bonus %']
bonus_df.head()


,EmployeeID,Salary,Band,Rating,Bonus %,Bonus
0,16000325,2000,1,4,1.2,4800.0
1,16041013,3102,1,4,1.2,7444.8
2,16150625,5283,1,4,1.2,12679.2
3,16150729,2000,1,4,1.2,4800.0
4,16240817,2000,1,4,1.2,4800.0


In [15]:
bonus_df['Bonus']=np.maximum(bonus_df['Bonus'],5000)
bonus_df.head()

,EmployeeID,Salary,Band,Rating,Bonus %,Bonus
0,16000325,2000,1,4,1.2,5000.0
1,16041013,3102,1,4,1.2,7444.8
2,16150625,5283,1,4,1.2,12679.2
3,16150729,2000,1,4,1.2,5000.0
4,16240817,2000,1,4,1.2,5000.0
